In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import json


import torch
import torchvision.models as models

# 定义与保存时一致的模型结构
class ModifiedResNet18(torch.nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedResNet18, self).__init__()
        self.resnet18 = models.resnet18(pretrained=False)  # 不使用预训练权重，因为我们加载的是已经训练好的模型
        self.resnet18.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet18.maxpool = torch.nn.Identity()  # 移除最大池化层
        
        # 获取ResNet18的特征提取部分
        self.features = torch.nn.Sequential(*list(self.resnet18.children())[:-1])
        
        # 添加MLP和全连接层
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5)
        )
        self.fc = torch.nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.mlp(x)
        x = self.fc(x)
        return x

# 初始化模型
model = ModifiedResNet18()

# 加载保存的模型权重
model_path = 'best_model.pth'  # 替换为你的模型文件路径
model.load_state_dict(torch.load(model_path))

# 如果你有GPU，可以将模型移动到GPU上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 设置模型为评估模式
model.eval()

# 现在你可以使用这个模型进行推理或其他操作

# 提取所有卷积层
conv_layers = []
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):
        conv_layers.append((name, module))

# 定义从本地读取部分CIFAR-10数据的预处理
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(data_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, self.image_files[idx])
        image = Image.open(img_name).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, 0  # 返回图像和虚拟标签0

def preprocess_local_cifar10(data_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    dataset = CustomDataset(data_path, transform=transform)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
    return dataloader

# 在process函数中，将输入数据移动到模型所在的设备上
def process(data_path, file_name):
    # 替换为你的部分CIFAR-10本地数据路径
    data_path = data_path
    dataloader = preprocess_local_cifar10(data_path)

    # 存储各层的激活值
    activations = {}

    # 定义钩子函数
    def get_activation(name):
        def hook(model, input, output):
            # 记录每个卷积核的平均激活强度
            activations[name] = output.detach().mean(dim=(0, 2, 3))  # [C] 维
        return hook

    # 为所有卷积层注册钩子
    handles = []
    for name, layer in conv_layers:
        handle = layer.register_forward_hook(get_activation(name))
        handles.append(handle)

    # 统计所有部分CIFAR-10数据的激活情况
    all_activation_stats = []

    # 遍历所有批次
    for batch_idx, (inputs, _) in enumerate(dataloader):
        inputs = inputs.to(device)  # 将输入数据移动到模型所在的设备上
        with torch.no_grad():
            model(inputs)

        # 定义激活阈值（例如：全局平均值的50%）
        all_activations = torch.cat([act.flatten() for act in activations.values()])
        threshold = all_activations.mean().item() * 0.5  # 可调参数
        threshold = 0

        # 统计每个卷积层的激活情况
        activation_stats = {}
        for layer_name, act in activations.items():
            activated_kernels = (act > threshold).nonzero().squeeze().tolist()
            if isinstance(activated_kernels, int):
                activated_kernels = [activated_kernels]
            # 按照降序排列激活索引
            activated_kernels.sort(reverse=True)
            activation_stats[layer_name] = {
                "total_kernels": len(act),
                "activated": len(activated_kernels),
                "indices": activated_kernels
            }

        all_activation_stats.append(activation_stats)

    # 移除钩子
    for handle in handles:
        handle.remove()

    # 保存结果到本地文件
    with open(file_name, "w") as f:
        json.dump(all_activation_stats, f)

    print("所有部分CIFAR-10数据的激活情况已保存到json 文件中")

    # 可视化部分（可选）
    # 如果需要可视化，可以参考之前的代码，但需要注意数据量较大，可能需要采样

D:\anaconda\envs\call\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\anaconda\envs\call\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [18]:
import json

# 读取 JSON 数据
with open(r'D:\python\activate\json\cifar_airplane_activation.json', 'r') as f:
    data = json.load(f)

# 初始化字典以存储每层的节点激活统计信息
layer_stats = {}

# 遍历每个数据集（多个字典对象）
for dataset in data:
    # 遍历每个层及其激活信息
    for layer_name, activation_info in dataset.items():
        total_kernels = activation_info['total_kernels']
        activated_indices = activation_info['indices']

        # 如果层尚未存在于统计字典中，则创建一个新条目
        if layer_name not in layer_stats:
            layer_stats[layer_name] = {
                "total_kernels": total_kernels,
                "index_counts": {}
            }

        # 统计每个索引的激活次数
        for index in activated_indices:
            if index not in layer_stats[layer_name]['index_counts']:
                layer_stats[layer_name]['index_counts'][index] = 0
            layer_stats[layer_name]['index_counts'][index] += 1

# 找出每个层中激活频率最高的节点
highest_activated_layers = []
for layer_name, stats in layer_stats.items():
    # 跳过下采样层（downsample）
    if 'downsample' in layer_name:
        continue

    # 获取激活次数最多的索引及其次数
    if stats['index_counts']:
        max_index = max(stats['index_counts'], key=stats['index_counts'].get)
        max_count = stats['index_counts'][max_index]
        highest_activated_layers.append({
            "layer_name": layer_name,
            "index": max_index,
            "total_activated": max_count,
            "total_kernels": stats['total_kernels']
        })

# 按照总激活次数从高到低排序
highest_activated_layers.sort(key=lambda x: x['total_activated'], reverse=True)

# 打印结果
for info in highest_activated_layers:
    print(f"Layer: {info['layer_name']}, Index: {info['index']}, "
          f"Total Activated: {info['total_activated']}, "
          f"Total Kernels: {info['total_kernels']}")

Layer: resnet18.layer1.0.conv1, Index: 26, Total Activated: 5897, Total Kernels: 64
Layer: resnet18.layer1.0.conv2, Index: 35, Total Activated: 5897, Total Kernels: 64
Layer: resnet18.layer1.1.conv1, Index: 48, Total Activated: 5897, Total Kernels: 64
Layer: resnet18.layer1.1.conv2, Index: 63, Total Activated: 5897, Total Kernels: 64
Layer: resnet18.layer2.0.conv1, Index: 101, Total Activated: 5897, Total Kernels: 128
Layer: resnet18.layer2.0.conv2, Index: 36, Total Activated: 5897, Total Kernels: 128
Layer: resnet18.layer2.1.conv1, Index: 97, Total Activated: 5897, Total Kernels: 128
Layer: resnet18.layer2.1.conv2, Index: 127, Total Activated: 5897, Total Kernels: 128
Layer: resnet18.layer3.0.conv1, Index: 209, Total Activated: 5897, Total Kernels: 256
Layer: resnet18.layer3.0.conv2, Index: 252, Total Activated: 5897, Total Kernels: 256
Layer: resnet18.layer3.1.conv1, Index: 230, Total Activated: 5897, Total Kernels: 256
Layer: resnet18.layer3.1.conv2, Index: 199, Total Activated: 589

In [20]:
import json

# 读取 JSON 数据
with open(r'D:\python\activate\json\cifar_airplane_activation.json', 'r') as f:
    data = json.load(f)

# 初始化字典以存储每层的节点激活统计信息
layer_stats = {}

# 遍历每个数据集（多个字典对象）
for dataset in data:
    # 遍历每个层及其激活信息
    for layer_name, activation_info in dataset.items():
        total_kernels = activation_info['total_kernels']
        activated_indices = activation_info['indices']

        # 如果层尚未存在于统计字典中，则创建一个新条目
        if layer_name not in layer_stats:
            layer_stats[layer_name] = {
                "total_kernels": total_kernels,
                "index_counts": {}
            }

        # 统计每个索引的激活次数
        for index in activated_indices:
            if index not in layer_stats[layer_name]['index_counts']:
                layer_stats[layer_name]['index_counts'][index] = 0
            layer_stats[layer_name]['index_counts'][index] += 1

# 找出每个层中激活频率最高的节点
highest_activated_layers = []
for layer_name, stats in layer_stats.items():
    # 跳过下采样层（downsample）
    if 'downsample' in layer_name:
        continue

    # 获取激活次数最多的索引及其次数
    if stats['index_counts']:
        max_index = max(stats['index_counts'], key=stats['index_counts'].get)
        max_count = stats['index_counts'][max_index]
        highest_activated_layers.append({
            "layer_name": layer_name,
            "index": max_index,
            "total_activated": max_count,
            "total_kernels": stats['total_kernels']
        })

# 按照总激活次数从高到低排序
highest_activated_layers.sort(key=lambda x: (x['total_activated'], -x['index']), reverse=True)

# 找出激活索引总体最靠前的一个节点
max_total_activated = highest_activated_layers[0]['total_activated']
closest_to_start = None

for info in highest_activated_layers:
    if info['total_activated'] == max_total_activated:
        if closest_to_start is None or info['index'] < closest_to_start['index']:
            closest_to_start = info
    else:
        break

# 打印结果
if closest_to_start:
    print(f"最高激活频率的层名：{closest_to_start['layer_name']}")
    print(f"激活索引：{closest_to_start['index']}")
    print(f"总激活次数：{closest_to_start['total_activated']}")
    print(f"总激活节点数：{closest_to_start['total_kernels']}")
else:
    print("未找到符合条件的节点")

最高激活频率的层名：resnet18.layer1.0.conv1
激活索引：26
总激活次数：5897
总激活节点数：64


In [2]:
path_list=[r"D:\python\resnetcifar\cifar10_sorted\cifar_airplane",
           r"D:\python\resnetcifar\cifar10_sorted\cifar_automobile",
          r"D:\python\resnetcifar\cifar10_sorted\cifar_bird",
          r"D:\python\resnetcifar\cifar10_sorted\cifar_cat",
          r"D:\python\resnetcifar\cifar10_sorted\cifar_deer",
          r"D:\python\resnetcifar\cifar10_sorted\cifar_dog",
          r"D:\python\resnetcifar\cifar10_sorted\cifar_frog",
          r"D:\python\resnetcifar\cifar10_sorted\cifar_horse",
          r"D:\python\resnetcifar\cifar10_sorted\cifar_ship",
          r"D:\python\resnetcifar\cifar10_sorted\cifar_truck"]

for i in path_list:
    filename=i.split("_")[-1]+"_activation.json"
    process(i,filename)

KeyboardInterrupt: 

In [8]:
import json

# 读取JSON文件
with open("airplane_activation.json", "r") as f:
    all_activation_stats = json.load(f)

# 显示激活统计信息
"""for batch_idx, activation_stats in enumerate(all_activation_stats):
    print(f"批次 {batch_idx + 1}:")
    for layer_name, stats in activation_stats.items():
        print(f"  层名: {layer_name}")
        print(f"    总卷积核数: {stats['total_kernels']}")
        print(f"    激活的卷积核数: {stats['activated']}")
        print(f"    激活的卷积核索引: {stats['indices'][:5]}...")  # 只显示前5个索引
    print("-" * 50)"""

'for batch_idx, activation_stats in enumerate(all_activation_stats):\n    print(f"批次 {batch_idx + 1}:")\n    for layer_name, stats in activation_stats.items():\n        print(f"  层名: {layer_name}")\n        print(f"    总卷积核数: {stats[\'total_kernels\']}")\n        print(f"    激活的卷积核数: {stats[\'activated\']}")\n        print(f"    激活的卷积核索引: {stats[\'indices\'][:5]}...")  # 只显示前5个索引\n    print("-" * 50)'

In [9]:
import json
from collections import defaultdict
def find_most_frequent_kernel(json_file):
    # 读取JSON文件
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    counts = defaultdict(lambda: defaultdict(int))  # 统计各层各索引的激活次数
    
    # 遍历每个图像的数据
    for image_data in data:
        for layer_name, stats in image_data.items():
            indices = stats['indices']
            for idx in indices:
                counts[layer_name][idx] += 1
    
    # 找出每个层的最高频率索引
    max_per_layer = {}
    for layer_name, indices in counts.items():
        if not indices:
            continue
        max_idx = max(indices, key=lambda k: indices[k])
        max_count = indices[max_idx]
        max_per_layer[layer_name] = (max_idx, max_count)
    
    # 找出全局最高
    global_max_layer = None
    global_max_idx = -1
    global_max_count = 0
    for layer_name, (idx, count) in max_per_layer.items():
        if count > global_max_count:
            global_max_count = count
            global_max_layer = layer_name
            global_max_idx = idx
    
    return global_max_layer, global_max_idx, global_max_count

In [15]:
categories = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
             'dog', 'frog', 'horse', 'ship', 'truck']

results = []
for category in categories:
    json_file = f"{category}_activation.json"
    try:
        layer, idx, count = find_most_frequent_kernel(json_file)
        results.append({
            '类别': category,
            '最高激活层': layer,
            '卷积核索引': idx,
            '激活次数': count
        })
    except FileNotFoundError:
        print(f"文件 {json_file} 不存在，跳过")
    except json.JSONDecodeError:
        print(f"文件 {json_file} 格式错误，跳过")

# 打印结果
for res in results:
    print(f"类别: {res['类别']}")
    print(f"  最高激活层: {res['最高激活层']}")
    print(f"  卷积核索引: {res['卷积核索引']}")
    print(f"  激活次数: {res['激活次数']}\n")

类别: airplane
  最高激活层: resnet18.layer1.0.conv1
  卷积核索引: 62
  激活次数: 5897

类别: automobile
  最高激活层: resnet18.layer1.0.conv1
  卷积核索引: 62
  激活次数: 5913

类别: bird
  最高激活层: resnet18.layer1.0.conv1
  卷积核索引: 62
  激活次数: 5891

类别: cat
  最高激活层: resnet18.layer1.0.conv1
  卷积核索引: 62
  激活次数: 5900

类别: deer
  最高激活层: layer1.0.conv1
  卷积核索引: 62
  激活次数: 5887

类别: dog
  最高激活层: layer1.0.conv1
  卷积核索引: 62
  激活次数: 5893

类别: frog
  最高激活层: layer1.0.conv1
  卷积核索引: 62
  激活次数: 5906

类别: horse
  最高激活层: layer1.0.conv1
  卷积核索引: 62
  激活次数: 5898

类别: ship
  最高激活层: layer1.0.conv1
  卷积核索引: 62
  激活次数: 5909

类别: truck
  最高激活层: layer1.0.conv1
  卷积核索引: 62
  激活次数: 5900



In [11]:
def get_layer_index(model, layer_name):
    for idx, layer in enumerate(model.modules()):
        if layer.__class__.__name__ == 'Sequential':
            for sub_layer in layer.modules():
                if sub_layer.__class__.__name__ != 'Sequential' and sub_layer.__class__.__name__ != 'ModuleList' and sub_layer.__class__.__name__ != 'ModuleDict':
                    if sub_layer.__class__.__name__ == layer_name.split('.')[-1]:
                        return idx
        else:
            if layer.__class__.__name__ == layer_name.split('.')[-1]:
                return idx
    return -1  # 如果找不到层，返回-1

In [13]:
layer_name = "layer1.0.conv1"
idx = get_layer_index(model, layer_name)
print(f"Layer index of '{layer_name}':", idx)

Layer index of 'layer1.0': -1
